# Q3 for HW 2 for CS 334 - Machine Learning Fall 2020
### By Alex Welsh

The first step is to import the necessary packages, datasets, and create a generic decision tree classifier and KNN

In [2]:
import argparse
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import time

In [3]:
# load the train and test data
xTrain = pd.read_csv("q4xTrain.csv")
yTrain = pd.read_csv("q4yTrain.csv")
xTest = pd.read_csv("q4xTest.csv")
yTest = pd.read_csv("q4yTest.csv")

In [35]:
dtClass = DecisionTreeClassifier()
knn = KNeighborsClassifier()

### Set up grid search for KNN

In [36]:
gridSearchKnn = GridSearchCV(knn, {'n_neighbors': range(1, 41, 2)})    # I tried K=1 to K=41
gridSearchKnn.fit(xTrain, yTrain.values.ravel())

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(1, 41, 2)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

### Grid Search on KNN, k=21 was the best parameter

In [52]:
print("Best parameters set found on development set:")              # I use the example code from class to print
print()                                                             # out the best parameters as well as the 
print(gridSearchKnn.best_score_, gridSearchKnn.best_params_)        # score for all the parameters
print()
print("Grid scores on development set:")
print()
means = gridSearchKnn.cv_results_['mean_test_score']
for mean, params in zip(means, gridSearchKnn.cv_results_['params']):
    print("%0.12f for %r" % (mean, params))
print()

Best parameters set found on development set:

0.8650624599615631 {'n_neighbors': 21}

Grid scores on development set:

0.842729019859 for {'n_neighbors': 1}
0.853459320948 for {'n_neighbors': 3}
0.855245035234 for {'n_neighbors': 5}
0.849863869315 for {'n_neighbors': 7}
0.857014734145 for {'n_neighbors': 9}
0.862375880846 for {'n_neighbors': 11}
0.864165598975 for {'n_neighbors': 13}
0.863272741832 for {'n_neighbors': 15}
0.862379884689 for {'n_neighbors': 17}
0.864169602819 for {'n_neighbors': 19}
0.865062459962 for {'n_neighbors': 21}
0.865062459962 for {'n_neighbors': 23}
0.865058456118 for {'n_neighbors': 25}
0.865058456118 for {'n_neighbors': 27}
0.865058456118 for {'n_neighbors': 29}
0.865058456118 for {'n_neighbors': 31}
0.864165598975 for {'n_neighbors': 33}
0.864165598975 for {'n_neighbors': 35}
0.864165598975 for {'n_neighbors': 37}
0.864165598975 for {'n_neighbors': 39}



### Grid search for decision tree classifer. Parameters included criteria (gini/entropy), max depth, and min leaf sample. The default cross validation value (cv=5) was used.

In [54]:
test_params = [{"criterion": ['gini', 'entropy'],                         # parameters for knn
        "max_depth": range(1, 21),
        "min_samples_leaf": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],}]
gridSearchDT = GridSearchCV(dtClass, test_params)
gridSearchDT.fit(xTrain, yTrain.values.ravel())                           # fit training values to grid search

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['gini', 'entropy'],
               

### For cv=5, the best parameters were gini, max_depth=3, and min_leaf_sample=1 with a score of 0.88563

In [55]:
print("Best parameters set found on development set:")                 # the same code was used before to print
print()                                                                # the best parameters and the same code  
print(gridSearchDT.best_score_, gridSearchDT.best_params_)             # is used for the other grid searches 
print()
print("Grid scores on development set:")
print()
means = gridSearchDT.cv_results_['mean_test_score']
for mean, params in zip(means, gridSearchDT.cv_results_['params']):
    print("%0.12f for %r" % (mean, params))
print()

Best parameters set found on development set:

0.8856302049967969 {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1}

Grid scores on development set:

0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 2}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 4}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 8}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 16}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 32}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 64}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 128}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 256}
0.864165598975 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 512}
0.878455317104 for {'criterion

### This grid search used the same parameters as before but tried using cv=10

In [56]:
test_params = [{"criterion": ['gini', 'entropy'],
        "max_depth": range(1, 21),
        "min_samples_leaf": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],}]
gridSearchDT = GridSearchCV(dtClass, test_params, cv=10)
gridSearchDT.fit(xTrain, yTrain.values.ravel())

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['gini', 'entropy'],
                 

### The best parameters for cv = 10 were entropy, max_depth=3, and min_leaf_sample=1 with a score of 0.88474

In [57]:
print("Best parameters set found on development set:")
print()
print(gridSearchDT.best_score_, gridSearchDT.best_params_)
print()
print("Grid scores on development set:")
print()
means = gridSearchDT.cv_results_['mean_test_score']
for mean, params in zip(means, gridSearchDT.cv_results_['params']):
    print("%0.12f for %r" % (mean, params))
print()

Best parameters set found on development set:

0.884740990990991 {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 1}

Grid scores on development set:

0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 2}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 4}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 8}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 16}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 32}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 64}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 128}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 256}
0.864165057915 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 512}
0.882054375804 for {'criteri

### This grid search used the same parameters as before but tried using cv=12. 12 was selected after trying different cv values and trying to find the cv value with the highest score.

In [68]:
test_params = [{"criterion": ['gini', 'entropy'],
        "max_depth": range(1, 21),
        "min_samples_leaf": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512],}]
gridSearchDT = GridSearchCV(dtClass, test_params, cv=12)
gridSearchDT.fit(xTrain, yTrain.values.ravel())

GridSearchCV(cv=12, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'criterion': ['gini', 'entropy'],
                 

### The best parameters for cv = 12 were entropy, max_depth=4, and min_leaf_sample=8 with a score of 0.88915

In [69]:
print("Best parameters set found on development set:")
print()
print(gridSearchDT.best_score_, gridSearchDT.best_params_)
print()
print("Grid scores on development set:")
print()
means = gridSearchDT.cv_results_['mean_test_score']
for mean, params in zip(means, gridSearchDT.cv_results_['params']):
    print("%0.12f for %r" % (mean, params))
print()

Best parameters set found on development set:

0.8891557995881948 {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 8}

Grid scores on development set:

0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 2}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 4}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 8}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 16}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 32}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 64}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 128}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 256}
0.864171051628 for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 512}
0.878441241516 for {'criter

# Best Parameters for Knn and Dt
**Best Params for Knn:** 0.8650624599615631 {'n_neighbors': 21}

**Best Params for DT:** 0.8891557995881948 {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 8}

Next, I created new datasets that dropped 5%, 10% and 20% of data by splitting the shuffled data. Then, I tested the classification of the KNN and DTClassifiers by fitting a KNN/DTClassifier to all the data, 5%, 10% and 20% dropped datasets. In each case, I used the optimal parameters found above to train the models.

In [18]:
trainX5, testXdrop5, trainY5, testYdrop5 = train_test_split(xTrain, yTrain, shuffle=True, test_size=.05)
trainX10, testXdrop10, trainY10, testYdrop10 = train_test_split(xTrain, yTrain, shuffle=True, test_size=.10)
trainX20, testXdrop20, trainY20, testYdrop20 = train_test_split(xTrain, yTrain, shuffle=True, test_size=.20)

In [19]:
knnAll = KNeighborsClassifier(n_neighbors=21)
knn5 = KNeighborsClassifier(n_neighbors=21)
knn10 = KNeighborsClassifier(n_neighbors=21)
knn20 = KNeighborsClassifier(n_neighbors=21)

In [20]:
knnAll.fit(xTrain, yTrain.values.ravel())
knn5.fit(trainX5, trainY5.values.ravel())
knn10.fit(trainX10, trainY10.values.ravel())
knn20.fit(trainX20, trainY20.values.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=21, p=2,
                     weights='uniform')

### Prints out acc and AUC for the 4 KNN's (KNNs trained on all data, 5% drop, 10% drop, and 20% drop)

They are then tested on the full test set

In [22]:
acc = knnAll.score(xTest,yTest)                               # get the accuracy score
yHatTest = knnAll.predict(xTest)                              # predict the labels with the model
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = knn5.score(xTest, yTest)
yHatTest = knn5.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = knn10.score(xTest,yTest)
yHatTest = knn10.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = knn20.score(xTest,yTest)
yHatTest = knn20.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)


Accuracy:  0.8604166666666667 Test AUC:  0.5040778498609824
Accuracy:  0.8645833333333334 Test AUC:  0.5129749768303985
Accuracy:  0.8666666666666667 Test AUC:  0.5076923076923077
Accuracy:  0.8625 Test AUC:  0.4987951807228916


### Set up the decision tree classifers with entropy, max_depth=4, and min_leaf_sample=8. 4 models trained on all data, 5% drop, 10% drop, and 20% dropped data.

In [23]:
dtAll = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=8)
dt5 = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=8)
dt10 = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=8)
dt20 = DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=8)

In [24]:
dtAll.fit(xTrain, yTrain.values.ravel())
dt5.fit(trainX5, trainY5.values.ravel())
dt10.fit(trainX10, trainY10.values.ravel())
dt20.fit(trainX20, trainY20.values.ravel())

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

### Accuracy and AUC are calculated by training the models on the full test set

In [25]:
acc = dtAll.score(xTest,yTest)                               # get the accuracy score
yHatTest = dtAll.predict(xTest)                              # predict the labels with the model
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = dt5.score(xTest, yTest)
yHatTest = dt5.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = dt10.score(xTest,yTest)
yHatTest = dt10.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

acc = dt20.score(xTest,yTest)
yHatTest = dt20.predict(xTest)
# calculate auc for test
testAuc = metrics.roc_auc_score(yTest['label'], yHatTest)
print("Accuracy: ", acc, "Test AUC: ", testAuc)

Accuracy:  0.86875 Test AUC:  0.5672845227062094
Accuracy:  0.8770833333333333 Test AUC:  0.6175162187210379
Accuracy:  0.8854166666666666 Test AUC:  0.6482854494902688
Accuracy:  0.875 Test AUC:  0.577386468952734


Final table of Accuracy and AUC for KNN's trained on all training data (KNN All), 5% dropped data (KNN 5%), 10% dropped data (KNN 10%), 20% dropped data (KNN 20%)

|       | Accuracy          | AUC               |
|-------|-------------------|-------------------|
|KNN All| 0.8604166666666667| 0.5040778498609824|
|KNN 5% | 0.8645833333333334| 0.5129749768303985|
|KNN 10%| 0.8666666666666667| 0.5076923076923077|
|KNN 20%| 0.8625            | 0.4987951807228916|

Final table of Accuracy and AUC for Decision Tree Classifers's trained on all training data (DTC All), 5% dropped data (DTC 5%), 10% dropped data (DTC 10%), 20% dropped data (DTC 20%)

|       | Accuracy          | AUC               |
|-------|-------------------|-------------------|
|DTC All| 0.86875           | 0.5672845227062094|
|DTC 5% | 0.8770833333333333| 0.6175162187210379|
|DTC 10%| 0.8854166666666666| 0.6482854494902688|
|DTC 20%| 0.875             | 0.577386468952734|